In [1]:
import os
import pandas as pd
import networkx as nx

In [3]:
ROOT_DATA = "../scraping/data"

# Data

In [6]:
city = "Berlin"

data = pd.read_csv(
    os.path.join(ROOT_DATA, "clean_data_tauschwohnung.csv"), index_col="Unnamed: 0"
)

subdata = data[(data["g-city"] == city) & (data["t-city"] == "Berlin")]

In [7]:
subdata

,g-city,g-rent,g-surf,g-rooms,t-city,t-max-rent,t-surf-min,t-rooms
0,Berlin,2500.0,145.0,5.0,Berlin,1400.0,72.0,2.0
1,Berlin,690.0,68.0,2.0,Berlin,1320.0,75.0,3.0
2,Berlin,1165.0,136.0,4.0,Berlin,800.0,50.0,NaN
3,Berlin,840.0,100.0,3.0,Berlin,500.0,40.0,2.0
4,Berlin,978.0,85.0,3.0,Berlin,1500.0,110.0,NaN
...,...,...,...,...,...,...,...,...
11607,Berlin,730.0,74.0,3.0,Berlin,2000.0,100.0,1.0
11609,Berlin,460.0,76.0,NaN,Berlin,1750.0,90.0,NaN
11610,Berlin,650.0,78.0,3.0,Berlin,900.0,80.0,4.0
11611,Berlin,300.0,72.0,2.0,Berlin,1000.0,94.0,3.0


In [20]:
graph = nx.Graph()

nodes_list = []
for idx, row in subdata.iterrows():
    nodes_list.append((idx, row.to_dict()))

In [21]:
graph.add_nodes_from(nodes_list)

# Shitty implementation with `itertools`

In [34]:
from itertools import combinations
from typing import Dict


def compatible_ads(ad1: Dict, ad2: Dict) -> bool:
    # Assumptions: one will accept the swap if nb rooms >= requested nb rooms
    if (ad1["g-city"] == ad2["t-city"]) and (ad2["g-city"] == ad1["t-city"]):
        if (ad1["g-rent"] <= ad2["t-max-rent"]) and (ad2["g-rent"] <= ad1["t-max-rent"]):
            if (ad1["g-surf"] >= ad2["t-surf-min"]) and (
                ad2["g-surf"] >= ad1["t-surf-min"]
            ):
                if (ad1["g-rooms"] >= ad2["t-rooms"]) and (
                    ad2["g-rooms"] >= ad1["t-rooms"]
                ):
                    return True

    return False


for idx1, idx2 in combinations(graph.nodes, r=2):
    if compatible_ads(graph.nodes[idx1], graph.nodes[idx2]):
        graph.add_edge(idx1, idx2)

KeyError: 't-rent'